In [1]:
import torch
import torchaudio
import polars as pl
import matplotlib.pyplot as plt
import os
import numpy as np

import helpers.input_processor as ip

/home/tiaan/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data from dataframe
data_dir = "data/raw_training/training_data/"
target_label = 'murmur_in_recording'
df = (
    ip.loadTrainingData(data_dir)
    .filter(pl.col(target_label) != 'Unknown')
    .pipe(ip.encodeData)
    .select([
        pl.col('audio_file').apply(lambda x: os.path.join(data_dir, x)),
        pl.col(target_label)
    ])
)

loading data from save file:  cache/ingested_data.json


In [3]:
#balance the data so that there is an equal number of murmur positive and murmur negative samples
#do this by duplicating random rows of whichever group (pos or neg) is smaller
neg_df = df.filter(pl.col(target_label)==0.0)
pos_df = df.filter(pl.col(target_label)==1.0)
numNeg = neg_df.height
numPos = pos_df.height

while numNeg != numPos:
    if numNeg < numPos:
        df.vstack(neg_df.sample(n=min(numPos-numNeg, neg_df.height), shuffle=True), in_place=True)
    else: 
        df.vstack(pos_df.sample(n=min(numNeg-numPos, pos_df.height), shuffle=True), in_place=True)
    numNeg = df.filter(pl.col(target_label)==0.0).height
    numPos = df.filter(pl.col(target_label)==1.0).height

#reshuffle rows
df = df.sample(frac=1.0, shuffle=True)

#check number of positive and negative samples
numNeg = df.filter(pl.col(target_label)==0.0).height
numPos = df.filter(pl.col(target_label)==1.0).height
print('Total Samples:       ', df.height)
print('Positive Samples:    ', numPos)
print('Negative Samples:    ', numNeg)
print('Percent Positive Samples:    ', numPos/(numPos+numNeg))

Total Samples:        5328
Positive Samples:     2664
Negative Samples:     2664
Percent Positive Samples:     0.5


In [4]:
import librosa
import math, random
import numpy as np

# x = path to audio file
# duration = length of time (in seconds) to which the signal is resized
# sr = sample rate of the signal
# **kwargs = keyword args to pass to librosa.feature.melspectrogram()
def preprocessAudio(x, duration, sr=4000, **kwargs):
    #read and load audio file in .wav format
    sig, samp_rate = librosa.load(x, sr=sr) #make sure that the correct sample rate is passed as a parameters. if unspecified, the function chooses some default value
    
    #resize sample, either by padding it with silence or truncating it
    sig = librosa.util.fix_length(sig, size=sr*duration)

    #implement audio augmentation:  ------------
    #time shift signal to the left or right by a random percent of its original length (max 99%)
    sig_len = sig.shape[0]
    max_shift = 0.99
    sig = np.roll(sig, round(random.random() * max_shift * sig_len))

    # #convert to stereo
    # sig = np.tile(sig, (2,1))
    #-------------------------------------------

    #get Mel spectrogram
    melSpec = librosa.feature.melspectrogram(y=sig, sr=samp_rate)
    melSpec = librosa.amplitude_to_db(melSpec)

    #implement image augmentation   ------------
    #-------------------------------------------

    return melSpec

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class AudioDataset(Dataset):
    def __init__(self, audioPaths, labels):
        self.audioPaths = audioPaths
        self.labels = labels
        self.sample_duration = 25
        self.n_mels = 128

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        spec = torch.from_numpy(preprocessAudio(self.audioPaths[idx], self.sample_duration, n_mels=self.n_mels))
        label = self.labels[idx]
        return spec, label

In [6]:
def splitDataset(ds, split_ratio=0.8):
    total_size = len(ds)
    train_size = round(split_ratio * total_size)
    test_size = total_size - train_size

    trainSet, testSet = torch.utils.data.random_split(ds, [train_size, test_size], generator=torch.Generator().manual_seed(0))
    return trainSet, testSet

In [7]:

audioPaths = df.get_column('audio_file').to_list()
labels = df.get_column(target_label).to_list()

ds = AudioDataset(audioPaths, labels)
train_ds, test_ds = splitDataset(ds)

# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

_________________________________________________________________________________________________________
tutorial code
_________________________________________________________________________________________________________

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)


dataiter = iter(train_loader)
images, labels = dataiter.next()

# images = torch.unsqueeze(images, 1)
images = images.unsqueeze(1)

x = 1
numClasses = 2
conv1 = nn.Conv2d(1, 32, 5)
pool = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(32, 64, 5)
print(images.shape)
y = conv1(images)
print(y.shape)
y = pool(y)
print(y.shape)
y = conv2(y)
print(y.shape)
y = pool(y)
print(y.shape)

torch.Size([4, 1, 128, 196])
torch.Size([4, 32, 124, 192])
torch.Size([4, 32, 62, 96])
torch.Size([4, 64, 58, 92])
torch.Size([4, 64, 29, 46])
